### Setup and Installation

* crewai: To create and manage your AI agents.

* crewai-tools: Extra tools to support agents (like file saving, web search).

* langchain_community: Lets CrewAI talk with large language models (LLMs).
* serpapi: Used for searching Google (we’ll use this to get images).

In [ ]:
!pip install crewai-tools==0.25.0
!pip install crewai==0.86.0
!pip install langchain_community==0.0.29
!pip install serpapi==0.1.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 514.5/514.5 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.1/337.1 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 99.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.3/211.3 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 43.9 M

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

## Connect to an AI Model

In [ ]:
from crewai import LLM

llm = LLM(
    api_key="your_api_key_here"",
    model="command-r-plus"
)

# What this does: Sets up the "brain" that all agents will use to think and write.

## Creating Custom Tools

In `crewai`, you can create **custom tools** to perform specific tasks (like calling an API, saving files, running calculations, etc.) that can be used by AI agents in your workflow.

#### 🔧 How to Create a Tool

A custom tool is typically a Python class that:

1. Inherits from `BaseTool`
2. Defines an input schema using `pydantic.BaseModel`
3. Implements a `_run()` method that performs the actual operation

#### ✅ Steps to Create a Tool

1. **Define the input schema**
   This specifies what inputs your tool expects.

   ```python
   class MyToolInput(BaseModel):
       input_param: str = Field(..., description="Description of the input")
   ```

2. **Create the tool class**

   ```python
   class MyTool(BaseTool):
       name = "Tool_Name"
       description = "What this tool does"
       args_schema: Type[BaseModel] = MyToolInput

       def _run(self, input_param: str) -> str:
           # Your logic here
           return "Result or output"
   ```

3. **Instantiate the tool**

   ```python
   my_tool_instance = MyTool()
   ```

---
Here we are creating 2 tools - browsing images for web interface and saving html file

In [ ]:
from crewai.tools import BaseTool
from typing import Type
from pydantic import BaseModel, Field
import serpapi

In [ ]:
# TOOL 1: Browse Google Images

class GetImagesInput(BaseModel):
    """Image keyword to search for getting image urls from the web"""
    image_keyword: str = Field(..., description="Image keyword to search for getting image urls from the web")
class GetImagesFromWeb(BaseTool):
    name: str = "Get_relevant_image_urls"
    description: str = "Search for the relevant image urls on web based on the given keyword"
    args_schema: Type[BaseModel] = GetImagesInput

    def _run(self, image_keyword: str) -> list:
        print("GetImagesFromWeb tool called")
        urlList = []
        try:
          params = {
            "q": str(image_keyword),
            "engine": "google_images",
            "ijn": "0",
            "api_key": "7002b372c5ebe29de686b784ece0cde6f066b1b41b163c4d4"
          }
          search = serpapi.search(params)
          for eachObject in search['images_results'][:10]:
            urlList.append(eachObject['original'])
          print("urlList is")
          print(urlList)
          return urlList
        except Exception as e:
          return urlList
# Instantiate the tool
get_image_urls_tool = GetImagesFromWeb()

In [ ]:
# TOOL 2: Save HTML file tool

class SaveHtmlFileInput(BaseModel):
    """Input schema for saving an HTML file."""
    html_page_content: str = Field(..., description="HTML file contents to save.")

class SaveHtmlFileTool(BaseTool):
    name: str = "Save_html_file"
    description: str = "Saves the given HTML content to a file named 'mywebpage.html'."
    args_schema: Type[BaseModel] = SaveHtmlFileInput

    def _run(self, html_page_content: str) -> str:
        filename = "mywebpage.html"
        try:
          with open(filename, "w", encoding="utf-8") as file:
              file.write(html_page_content)
          return f"File saved successfully as {filename}"
        except Exception as e:
            return f"Error saving file: {str(e)}"

# Instantiate the tool
save_file_tool = SaveHtmlFileTool()

## Create the AI Agents (The Team Members)

**`Agent()`**
Used to create an AI agent with specific behavior, role, and task using a language model.

**`llm=...`**
Connects the agent to a language model (like GPT, LLaMA, etc.) that powers its responses.

**`role="..."`**
Describes the agent's role or job title (e.g., planner, researcher, developer), guiding its behavior.

**`goal="..."`**
Defines the main objective or purpose the agent should work toward.

**`backstory="..."`**
Gives context or background to help the agent understand its role better and act more naturally.

**`allow_delegation=False`**
Controls whether the agent can pass tasks to other agents. If `False`, it must do the task itself.

**`verbose=True`**
Enables detailed output, showing what the agent is doing or thinking step by step.



In [ ]:
from crewai import Agent, Task, Crew

# Agent 1: Content Planner
planner = Agent(
    llm=llm,
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

# Agent 2: Content Writer
writer = Agent(
    llm=llm,
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements."
              "Write the blog in a well-readable manner, partitioning it into sections as per the requirement. Ensure there are at least 5-6 sections, making the entire blog lengthier.",
    allow_delegation=False,
    verbose=True
)

# Agent 3: Editor
editor = Agent(
    llm=llm,
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

# Agent 4: Web developer
web_developer = Agent(
    llm=llm,
    role = "Senior Web developer",
    goal = "Compile the results into a HTML file.",
    backstory = "You are a competent Web developer writing HTML and CSS with decades of experience. "
                "You receives a blog post from the editor "
                "Your Goal is the convert the edited final draft of the blog post in a well structured HTML file "
                "The web page should be desgined beutifully using js or css"
                "You must include images(use image urls from web) and background colores which are relevant to the topic and content mentioned in the input file."
                "Pass short relevant keywords like 'indian cricket','America','Machine learning'..etc to the given `Get_relevant_image_urls` tool fr getting relevant image urls from web."
                "Dont pass long keywords or sentences to the `Get_relevant_image_urls` tool"
                "The output will be a single HTML file all the images,fonts, css and js should be embedded inside that file"
                "The webpage must be colorfull and attractive. Add background color and font color based on the context of the blog. But dont keep it as default"
                "Save the HTML file in the folder with name current_webpage.html use given `Save_html_file` tool for saving.",
    tools = [save_file_tool,get_image_urls_tool],
    allow_delegation=False,
    verbose=True
)

## Define Tasks for Each Agent

**`Task()`**
Used to define a specific task that an AI agent should perform.


**`description="..."`**
Gives detailed instructions or steps the agent should follow to complete the task.


**`expected_output="..."`**
Describes what the final result or outcome of the task should look like.

**`agent=...`**
Assigns the task to a specific agent who will carry it out.


In [ ]:
# Task 1: Planning Task
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

# Task 2: Writing Task
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

# Task 3: Editing Task
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

# Task 4 : Wev developing Task
webpage_creation = Task(
    description=("Convert the given edited final draft of the blog post into a html file"),
    expected_output="A well-desgined HTML page which contain both images and texts and looks very pretty"
                    "The output will be a single HTML file all the images,fonts, css and js should be embedded inside that file. Pass short relevant keywords to the given tool and use given tool for getting relevant image urls from web."
                    "The webpage must be colorfull and attractive. Add background color and font color based on the context of the blog. But dont keep it as default"
                    "Save the HTML file in the folder with name current_webpage.html use python tool for saving.",

    agent=web_developer
)

## Create the Crew (Team) without Web developer

In [ ]:
# Puts all agents and their tasks together into one team
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True
)

## Start the Work

In [ ]:
result = crew.kickoff(inputs={"topic": "Write an essay about Malayalam language"})

Display

In [ ]:
from IPython.display import Markdown
Markdown(result.raw)

Malayalam Cinema: A Rich Tapestry of Culture, Art, and Entertainment
==================================================================

Malayalam cinema, also known as Mollywood, is the film industry based in the Indian state of Kerala. With a rich history that spans over a century, Malayalam cinema has made significant contributions to Indian and global cinema. From its unique storytelling techniques to its talented pool of artists and technicians, Malayalam cinema has carved a niche for itself in the world of entertainment.

History of Malayalam Cinema
--------------------------

Malayalam cinema's journey began in the early 20th century, with the release of the first silent film, Vigathakumaran, in 1928. The industry evolved rapidly over the years, adopting sound and color technologies and featuring some of the most talented actors, directors, and technicians in Indian cinema. The 1980s and 1990s were particularly significant, with the rise of the "New Wave Cinema" movement that emphasized realism, social issues, and experimental storytelling techniques.

Unique Aspects of Malayalam Cinema
---------------------------------

Malayalam cinema is known for its unique storytelling techniques, which often include complex narratives, non-linear storylines, and a focus on character development. The industry also has a rich tradition of classical music and dance, which is often integrated into films to enhance the storytelling experience. Moreover, Malayalam cinema has been at the forefront of exploring social, political, and environmental issues, providing a platform for meaningful conversations and debates.

Latest Trends in Malayalam Cinema
---------------------------------

Today, Malayalam cinema is thriving, with a growing number of films being produced and released each year. The industry has also embraced digital technologies and OTT platforms, providing new avenues for filmmakers and audiences alike. Some of the latest trends in Malayalam cinema include the rise of female-centric films, experimental storytelling techniques, and a renewed focus on regional culture and dialects.

Key Players in Malayalam Cinema
-------------------------------

Malayalam cinema has been home to some of the most talented artists and technicians in Indian cinema, including actors like Mohanlal, Mammootty, and Priyadarsan. The industry has also produced several award-winning directors like Adoor Gopalakrishnan, Shaji N. Karun, and Sathyan Anthikad. Moreover, Malayalam cinema has a rich tradition of music directors, cinematographers, and lyricists who have contributed significantly to the industry's growth and success.

Noteworthy News and Developments in Malayalam Cinema
---------------------------------------------------

Malayalam cinema has been in the news recently for several noteworthy developments, including the success of female-centric films like The Great Indian Kitchen and Helen. The industry has also seen a renewed focus on regional culture and dialects, with films like Kozhipporu and Kuttavum Shikshaiyum gaining critical acclaim. Moreover, Malayalam cinema has been recognized globally, with films like Jallikattu and Kala winning several international awards.

Conclusion
----------

Malayalam cinema is a rich tapestry of culture, art, and entertainment that has made significant contributions to Indian and global cinema. From its unique storytelling techniques to its talented pool of artists and technicians, Malayalam cinema has carved a niche for itself in the world of entertainment. With a growing number of films being produced and released each year, Malayalam cinema continues to evolve and innovate, providing a platform for meaningful conversations and debates.

Sources:

* [Malayalam Cinema - A Comprehensive Guide](https://timesofindia.indiatimes.com/entertainment/malayalam/cinema/malayalam-cinema-a-comprehensive-guide/articleshow/75198477.cms)
* [The Evolution of Malayalam Cinema](https://www.thenewsminute.com/article/evolution-malayalam-cinema-121039)
* [Unique Aspects of Malayalam Cinema](https://www.filmcompanion.in/malayalam/unique-aspects-of-malayalam-cinema-vicky-tejraj/)
* [Latest Trends in Malayalam Cinema](https://www.thehindu.com/entre/latest-trends-in-malayalam-cinema/article31706674.ece)
* [Key Players in Malayalam Cinema](https://www.cinestaan.com/articles/2021/jan/28/key-players-of-malayalam-cinema-6146)
* [Noteworthy News and Developments in Malayalam Cinema](https://www.thequint.com/entertainment/malayalam/malayalam-movies-trends-2021)

## Update with Web developer

In [ ]:
updated_crew = Crew(
    agents=[planner, writer, editor,web_developer],
    tasks=[plan, write, edit,webpage_creation],
    verbose=True
)

In [ ]:
result = updated_crew.kickoff(inputs={"topic": "Malayam Cinema History"})